In [69]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [70]:
import psycopg2
import csv

# Replace these variables with your actual database connection details
hostname = 'ec2-3-9-191-104.eu-west-2.compute.amazonaws.com'
database = 'testdb'
username = 'consultants'
password = 'WelcomeItc@2022'
port_id = 5432

# Name of the local 'csv' file
txtfile = "zz_seb_table_1"
# Set table name
tableName = f"public.{txtfile}"

# Corrected path to your CSV file for Windows
# csv_file_path = r"C:\Users\sebou\demo\ITC\ITC\HADOOP\seb_table_0.txt" 
csv_file_path = r"C:\Users\sebou\demo\ITC\ITC\HADOOP\zz_seb_table_1.txt" 

try:
    # Establishing the connection
    conn = psycopg2.connect(
        host=hostname,
        dbname=database,
        user=username,
        password=password,
        port=port_id
    )
    conn.autocommit = False  # Start transaction control
    cursor = conn.cursor()


    # Create a table with 'id' as the PRIMARY KEY if it doesn't exist
    create_table_query = f"""
    CREATE TABLE IF NOT EXISTS {tableName} (
        id INT PRIMARY KEY,
        name VARCHAR(255) NOT NULL,
        age INT NOT NULL
    );
    """
    cursor.execute(create_table_query)

    # Open the CSV file and skip the header row
    with open(csv_file_path, 'r') as csv_file:
        csv_reader = csv.DictReader(csv_file)  # Using DictReader to handle columns by name

        for row in csv_reader:
            # Upsert query using ON CONFLICT clause
            upsert_query = f"""
            INSERT INTO {tableName} (id, name, age) VALUES (%s, %s, %s)
            ON CONFLICT (id) DO UPDATE
            SET name = EXCLUDED.name, age = EXCLUDED.age;
            """
            cursor.execute(upsert_query, (row['id'], row['name'], row['age']))

    # Commit the transaction
    conn.commit()
    print("Table created (if not existed) and data upserted successfully!")

except psycopg2.Error as e:
    print("Database error:", e)
    conn.rollback()  # Rollback in case of any error

finally:
    # Closing the cursor and connection
    if cursor:
        cursor.close()
    if conn:
        conn.close()

        

Table created (if not existed) and data upserted successfully!


In [85]:
# tableName = f"public.{txtfile}"
tableName = f"public.zz_seb_table_1"

In [86]:
import psycopg2
from sqlalchemy import create_engine

# Replace these variables with your actual database connection details
hostname = 'ec2-3-9-191-104.eu-west-2.compute.amazonaws.com'
username = 'consultants'
password = 'WelcomeItc@2022'
port_id = 5432
database = 'testdb'

# Correctly URL-encoded password in the database URL
database_url = ('postgresql://'
                f'{username}:'
                'WelcomeItc%402022'
                f'@{hostname}'
                f':{port_id}'
                f'/{database}' )

engine = create_engine(database_url)

# To execute a query and fetch results with SQLAlchemy
# tableName = "your_table_name"  # Replace with your actual table name
with engine.connect() as connection:
    result = connection.execute(f"SELECT * FROM {tableName} ;")
    for row in result:
        print(row)


(1, 'John Doe', 30)
(2, 'Jane Smith', 32)
(3, 'Alex Johnson', 25)
(4, 'Maria Garcia', 28)
(5, 'Chen Wei', 33)
(6, 'Olivia Brown', 29)
(7, 'Liam Miller', 31)
(8, 'Emma Wilson', 27)
(9, 'Noah Davis', 34)
(10, 'Ava Taylor', 26)
(11, 'aJohn Doe', 30)


In [ ]:
# Open the permissions widely on the warehouse
sudo -u hdfs hdfs dfs -chmod -R 777 /tmp/USUK30/seb/hive/the_z_file/
sudo -u hdfs hdfs dfs -chown -R hive:hive /tmp/USUK30/seb/hive/the_z_file/


In [ ]:
# On Hive, find the max_id 
# Store it into a variable and next cell:

In [ ]:
last_value=$(beeline -u 'jdbc:hive2://ip-172-31-3-80.eu-west-2.compute.internal:10000/default;' --silent=true -e "SELECT MAX(id) FROM ze_table;" | grep -o '[0-9]*' | tail -n 1)
echo $last_value

--last-value $last_value \

In [ ]:
#!/bin/bash

### This works and does the incremental load 

sqoop import \
    --connect jdbc:postgresql://ec2-3-9-191-104.eu-west-2.compute.amazonaws.com:5432/testdb \
    --username consultants \
    --password WelcomeItc@2022 \
    --query "SELECT * FROM public.ze_table WHERE id > 19 AND \$CONDITIONS" \
    --target-dir /tmp/USUK30/seb/hive/import_23/ \
    --split-by id \
    --as-textfile \
    --incremental append \
    --check-column id \
    --last-value 19



In [ ]:
Automating the Workflow
For automation, consider scripting the entire process or using orchestration tools like Apache Airflow. You can set up a DAG (Directed Acyclic Graph) that:

Watches for changes in the CSV file.
Updates the PostgreSQL database.
Runs the Sqoop job to update Hive.

In [43]:
import psycopg2

# Replace these variables with your actual database connection details
host = "ec2-3-9-191-104.eu-west-2.compute.amazonaws.com"
dbname = "testdb"
user = "consultants"  # Use your master user with privileges to create roles and grant permissions
password = "WelcomeItc@2022"
port = "5432"

conn_string = f"host={host} dbname={dbname} user={user} password={password} port={port}"

try:
    # Connect to the PostgreSQL server
    conn = psycopg2.connect(conn_string)
    print("Connected to the database.")
    cur = conn.cursor()
    
    # Create role
    cur.execute("CREATE ROLE consultants LOGIN PASSWORD 'WelcomeItc@2022';")
    
    # Grant SELECT permission on the table for import operations
    cur.execute("GRANT SELECT ON seb_table TO consultants;")
    
    # Grant additional permissions as needed
    cur.execute("GRANT INSERT, UPDATE, DELETE ON seb_table TO consultants;")
    
    # Optionally grant CREATE on the database
    cur.execute("GRANT CREATE ON DATABASE your_database_name TO consultants;")

    # Commit the transaction
    conn.commit()
    print("Role created and permissions granted.")
    
except Exception as e:
    print(f"An error occurred: {e}")
finally:
    # Close the connection
    if conn:
        conn.close()
        print("Database connection closed.")


Connected to the database.
An error occurred: permission denied to create role

Database connection closed.


In [44]:
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

# Database connection parameters
hostname = 'ec2-3-9-191-104.eu-west-2.compute.amazonaws.com'
dbname = 'testdb'
username = 'consultants'
password = 'WelcomeItc@2022'
port_id = 5432

# Connect to PostgreSQL DB as a superuser or a user with necessary privileges
try:
    conn = psycopg2.connect(
        host=hostname,
        dbname=dbname,
        user=username,
        password=password,
        port=port_id
    )
    conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)  # Required for executing CREATE DATABASE command
    cursor = conn.cursor()

    # Commands to execute
    create_role_command = "CREATE ROLE new_role LOGIN PASSWORD 'password';"
    grant_privileges_command = "GRANT ALL PRIVILEGES ON DATABASE testdb TO new_role;"

    # Execute commands
    cursor.execute(create_role_command)
    cursor.execute(grant_privileges_command)
    
    print("Role created and privileges granted successfully.")

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    cursor.close()
    conn.close()


An error occurred: permission denied to create role



In [ ]:
sqoop import \
--connect jdbc:postgresql://ec2-3-9-191-104.eu-west-2.compute.amazonaws.com:5432/testdb \
--username consultants --password WelcomeItc@2022 \
--table public.seb_table \  # Ensure this matches your PostgreSQL table name
--hive-import --hive-table yadb.seb_table \  # Adjust the Hive database and table name as needed
--target-dir /user/hive/warehouse/default.db/seb_table \  # Adjust the target directory as needed
--delete-target-dir \  # This deletes the target directory before importing
--fields-terminated-by "," \
-m 1 \
--hs2-url "jdbc:hive2://ip-172-31-3-80.eu-west-2.compute.internal:10000/default;"


In [ ]:
sqoop import \
--connect jdbc:postgresql://ec2-3-9-191-104.eu-west-2.compute.amazonaws.com:5432/testdb \
--username consultants --password WelcomeItc@2022 \
--table public.seb_table \  # Ensure this matches your PostgreSQL table name
--hive-import --hive-table seb_table_on_hive_2 \  # Adjust the Hive database and table name as needed
--target-dir /user/hive/warehouse/default.db/seb_table_on_hive_2 \  # Adjust the target directory as needed
--delete-target-dir \  # This deletes the target directory before importing
--fields-terminated-by "," \
-m 1 \
--hs2-url "jdbc:hive2://ip-172-31-3-80.eu-west-2.compute.internal:10000/default;"


In [ ]:
sqoop import \
  --connect jdbc:postgresql://ec2-3-9-191-104.eu-west-2.compute.amazonaws.com:5432/testdb \
  --username consultants \
  --password WelcomeItc@2022 \
  --table public.seb_table \
  --hive-import \
  --hive-database default \
  --hive-table seb_table_on_hive \
  --incremental append \
  --check-column id \
  --last-value 20 \
  -m 1 \
  --hs2-url "jdbc:hive2://ip-172-31-3-80.eu-west-2.compute.internal:10000/default"


In [ ]:
sqoop import \
  --connect jdbc:postgresql://ec2-3-9-191-104.eu-west-2.compute.amazonaws.com:5432/testdb \
  --username consultants \
  --password WelcomeItc@2022 \
  --table seb_table \
  --hive-import \
  --hive-database default \
  --hive-table seb_table_on_hive \
  --incremental append \
  --check-column id \
  --last-value 20 \
  -m 1 \
  --hs2-url "jdbc:hive2://ip-172-31-3-80.eu-west-2.compute.internal:10000/default"


In [ ]:
sqoop import \
  --connect jdbc:postgresql://ec2-3-9-191-104.eu-west-2.compute.amazonaws.com:5432/testdb \
  --username consultants \
  --password WelcomeItc@2022 \
  --table seb_table \
  --hive-import \
  --hive-database default \
  --hive-table seb_table_on_hive \
  --incremental append \
  --check-column id \
  --last-value 20 \
  -m 1 \
  --hs2-url "jdbc:hive2://ip-172-31-3-80.eu-west-2.compute.internal:10000/default"


In [ ]:
sqoop import \
--connect jdbc:postgresql://ec2-3-9-191-104.eu-west-2.compute.amazonaws.com:5432/testdb \
--username consultants \
--password WelcomeItc@2022 \
--table seb_table \
--hive-import \
--hive-database default \
--hive-table seb_table_on_hive \
--incremental append \
--check-column id \
--last-value 20 \
-m 1 \
--hs2-url "jdbc:hive2://ip-172-31-3-80.eu-west-2.compute.internal:10000/default"

In [ ]:
sqoop import \
--connect jdbc:postgresql://ec2-3-9-191-104.eu-west-2.compute.amazonaws.com:5432/testdb \
--username consultants \
--password WelcomeItc@2022 \
--table seb_table \
--hive-import \
--hive-database yadb \
--hive-table seb_table_on_hive_2 \
--incremental append \
--check-column id \
--last-value 26 \
-m 1 \
--hs2-url "jdbc:hive2://ip-172-31-3-80.eu-west-2.compute.internal:10000/yadb"

In [ ]:
sqoop import --connect jdbc:postgresql://ec2-3-9-191-104.eu-west-2.compute.amazonaws.com:5432/testdb --username consultants --password WelcomeItc@2022 --query 'SELECT * FROM public.person WHERE id > 7 AND $CONDITIONS' --target-dir /tmp/USUK30/Asad/import/ --split-by id --as-textfile --incremental append --check-column id --last-value 112